In [42]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score, balanced_accuracy_score, precision_recall_fscore_support, classification_report
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np

import os
os.environ["WANDB_PROJECT"] = "offline"

training_directory ='fewshot'
modname = "mlburnham/Political_DEBATE_base_v1.0"
# instantiate model
model = AutoModelForSequenceClassification.from_pretrained(modname, num_labels = 2, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(modname)

def metrics(df, preds, group_by=None):
    true_col = 'entailment'
    
    def get_metrics(y_true, y_pred):
        return {
            'MCC': matthews_corrcoef(y_true, y_pred),
            'Accuracy': accuracy_score(y_true, y_pred),
            'F1': f1_score(y_true, y_pred, average='weighted')
        }
    
    results = []
    
    if group_by not in ['dataset', 'task']:
        for col in preds:
            metrics = get_metrics(df[true_col], df[col])
            metrics['Column'] = col
            results.append(metrics)
    else:
        for col in preds:
            for group_name, group in df.groupby(group_by):
                metrics = get_metrics(group[true_col], group[col])
                metrics['Column'] = col
                metrics[group_by.capitalize()] = group_name
                results.append(metrics)
    
    results_df = pd.DataFrame(results)
    
    if group_by in ['dataset', 'task']:
        return results_df.set_index(['Column', group_by.capitalize()])
    else:
        return results_df.set_index('Column')

def truncate(text):
    words = text.split()
    if len(words) > 450:
        return " ".join(words[:450])
    return text


def tokenize_function(docs):
    return tokenizer(docs['premise'], docs['hypothesis'], padding = 'max_length', truncation = True)


def compute_metrics_standard(eval_pred, label_text_alphabetical=list(model.config.id2label.values())):
    labels = eval_pred.label_ids
    pred_logits = eval_pred.predictions
    preds_max = np.argmax(pred_logits, axis=1)

    # metrics
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro') 
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds_max, average='micro')
    acc_balanced = balanced_accuracy_score(labels, preds_max)
    acc_not_balanced = accuracy_score(labels, preds_max)
    mcc = matthews_corrcoef(labels, preds_max)

    metrics = {'MCC': mcc,
            'f1_macro': f1_macro,
            'f1_micro': f1_micro,
            'accuracy_balanced': acc_balanced,
            'accuracy': acc_not_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'precision_micro': precision_micro,
            'recall_micro': recall_micro,
            }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} )
    print("Detailed metrics: ", classification_report(
        labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical, sample_weight=None,
        digits=2, output_dict=True, zero_division='warn'),
    "\n")

    return metrics

# Freedom and Rights

In [52]:
fr = pd.read_csv('freedom_test.csv')
fr = fr[~fr['text'].isna()]
fr['text'] = fr['text'].astype(str)
fr['hypothesis'] = 'This text is about freedom and rights.'
fr.rename({'text':'premise', 'freedom_and_rights':'entailment'}, inplace = True, axis = 1)
fr.drop_duplicates('premise', inplace = True)
fr['entailment'].replace({1:0, 0:1}, inplace = True)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\886333745.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fr['entailment'].replace({1:0, 0:1}, inplace = True)


### Few Shot

In [53]:
training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    lr_scheduler_type= "linear",
    group_by_length=False,
    learning_rate = 2e-5,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 1, 
    num_train_epochs=5,
    warmup_ratio=0.06,  
    weight_decay=0.01, 
    fp16=True,   
    fp16_full_eval=True,
    eval_strategy="no",
    seed=1,
    save_strategy="no",
    dataloader_num_workers = 12,
)

tokenizer = AutoTokenizer.from_pretrained(modname)

In [54]:
%%time
# Define a function to initialize the model in the trainer. This will make results reproducible
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modname, num_labels = 2, ignore_mismatched_sizes=True)

# Define the number of samples (shots) and random seeds to use
shots = [10, 25, 50, 100]
seeds = range(1,11)

# Initialize lists to store results
mcc_list = []
acc_list = []
shots_list = []

# Iterate through different shot sizes
for shot in shots:
    # Iterate through different random seeds
    for seed in seeds:
        # Sample training data based on current shot size and seed
        train = fr.sample(shot, random_state = seed)
        # Create validation set with remaining instances
        val = fr[~fr.index.isin(train.index)]
        
        # Create a DatasetDict with train and validation splits
        ds = DatasetDict({'train': Dataset.from_pandas(train, preserve_index=False), 'validation':Dataset.from_pandas(val, preserve_index=False)})
        # Tokenize the dataset
        dstok = ds.map(tokenize_function, batched = True)
        # Rename 'entailment' column to 'label'
        dstok = dstok.rename_columns({'entailment':'label'})
        # Define label mapping
        id2label = {0: "entailment", 1: "not_entailment"}
        
        # Initialize the Trainer
        trainer = Trainer(
            model_init = model_init,
            tokenizer=tokenizer,
            args=training_args,
            train_dataset=dstok['train'],
            eval_dataset=dstok['validation'],
            compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(model.config.id2label.values()))
        )
        
        # Train the model
        trainer.train()
        # Make predictions on validation set
        res = trainer.predict(dstok['validation'])
        preds = np.argmax(res.predictions, axis=-1)
        
        # Calculate Matthews Correlation Coefficient
        mcc_res = matthews_corrcoef(val['entailment'], preds)
        mcc_list.append(mcc_res)
        # Calculate Accuracy
        acc_res = accuracy_score(val['entailment'], preds)
        acc_list.append(acc_res)
        # Store the current shot size
        shots_list.append(shot)
    
    # Print progress
    print(shot)

Map: 100%|██████████| 7620/7620 [00:01<00:00, 4660.67 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Aggregate metrics:  {'MCC': 0.701119785385075, 'f1_macro': 0.8504231878390929, 'f1_micro': 0.8682414698162729, 'accuracy_balanced': 0.8474726076273464, 'accuracy': 0.8682414698162729, 'precision_macro': 0.8536746082050919, 'recall_macro': 0.8474726076273464, 'precision_micro': 0.8682414698162729, 'recall_micro': 0.8682414698162729}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\1915898394.py:87: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.8128058727569332, 'recall': 0.7852639873916469, 'f1-score': 0.7987975951903807, 'support': 2538.0}, 'not_entailment': {'precision': 0.8945433436532507, 'recall': 0.9096812278630461, 'f1-score': 0.9020487804878049, 'support': 5082.0}, 'accuracy': 0.8682414698162729, 'macro avg': {'precision': 0.8536746082050919, 'recall': 0.8474726076273464, 'f1-score': 0.8504231878390929, 'support': 7620.0}, 'weighted avg': {'precision': 0.8673189734255796, 'recall': 0.8682414698162729, 'f1-score': 0.8676588187706313, 'support': 7620.0}} 



...

Map: 100%|██████████| 7530/7530 [00:01<00:00, 4828.91 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Aggregate metrics:  {'MCC': 0.7889229456296178, 'f1_macro': 0.8923245351693465, 'f1_micro': 0.9074369189907039, 'accuracy_balanced': 0.8801260786485633, 'accuracy': 0.9074369189907039, 'precision_macro': 0.9093374863635296, 'recall_macro': 0.8801260786485633, 'precision_micro': 0.9074369189907039, 'recall_micro': 0.9074369189907039}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\1915898394.py:87: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.9138952164009112, 'recall': 0.7979315831344471, 'f1-score': 0.851985559566787, 'support': 2514.0}, 'not_entailment': {'precision': 0.904779756326148, 'recall': 0.9623205741626795, 'f1-score': 0.9326635107719061, 'support': 5016.0}, 'accuracy': 0.9074369189907039, 'macro avg': {'precision': 0.9093374863635296, 'recall': 0.8801260786485633, 'f1-score': 0.8923245351693465, 'support': 7530.0}, 'weighted avg': {'precision': 0.9078230852276027, 'recall': 0.9074369189907039, 'f1-score': 0.9057280035568106, 'support': 7530.0}} 

100
CPU times: total: 39min 45s
Wall time: 3h 44min 16s


In [59]:
res_fr = pd.DataFrame({'n':shots_list, 'mcc':mcc_list})
#res_fr['mcc'] = acc_list
res_fr.groupby('n').mean()

,mcc
n,
10,0.722883
25,0.737884
50,0.760218
100,0.780575


### Zero Shot

In [61]:
pipe = pipeline("zero-shot-classification", model = modname, device = 0, batch_size = 32)
res = pipe(list(fr['premise'].str.lower()), ['freedom and rights except voting.'], hypothesis_template = 'This text is about {}', multi_label = False)
labels = [round(label['scores'][0], 0) for label in res]
fr['0_shot'] = labels
fr['0_shot'].replace({0:1, 1:0}, inplace = True)

zs_fr = pd.DataFrame({'n':0, 'mcc':matthews_corrcoef(fr['entailment'], fr['0_shot']), 'accuracy':accuracy_score(fr['entailment'], fr['0_shot'])}, index = [0])
res_fr = pd.concat([res_fr, zs_fr], axis = 0)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\3963122802.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  fr['0_shot'].replace({0:1, 1:0}, inplace = True)


In [62]:
res_fr.to_csv('motn_fewshot_base2.csv', index = False)

### Supervised

In [53]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report, matthews_corrcoef

In [63]:
fr = pd.read_csv('freedom_test.csv')
fr = fr[~fr['text'].isna()]
fr['text'] = fr['text'].astype(str)
fr.drop_duplicates('text', inplace = True)
fr.rename({'freedom_and_rights': 'label'}, axis = 1, inplace = True)

In [64]:
val = fr.sample(frac = .2, random_state = 1)
train = fr[~fr.index.isin(val.index)]

train = fr.sample(100, random_state = 1)
val = fr[~fr.index.isin(train.index)]

In [94]:
#val = df.sample(frac = .2, random_state = 1)
#train = df[~df.index.isin(val.index)]

train = df.sample(50, random_state = 1)
val = df[~df.index.isin(train.index)]

In [95]:
modname = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(modname)

In [96]:
ds = DatasetDict({'train': Dataset.from_pandas(train, preserve_index=False), 'validation':Dataset.from_pandas(val, preserve_index=False)})

In [97]:
ds = ds.rename_columns({'premise':'text', 'entailment':'label'})

In [98]:
def tokenize_function(docs):
    return tokenizer(docs['text'], padding = True, truncation = True)

dstok = ds.map(tokenize_function, batched = True)
model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels = 2, ignore_mismatched_sizes=True)

Map: 100%|██████████| 1937/1937 [00:00<00:00, 5374.18 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    #deepspeed="ds_config_zero3.json",  # if using deepspeed
    lr_scheduler_type= "linear",
    group_by_length=True,  # can increase speed with dynamic padding, by grouping similar length texts https://huggingface.co/transformers/main_classes/trainer.html
    learning_rate=9e-6 if "large" in modname else 2e-5,
    per_device_train_batch_size=4 if "large" in modname else 2,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4 if "large" in modname else 1,  # (!adapt/halve batch size accordingly). accumulates gradients over X steps, only then backward/update. decreases memory usage, but also slightly speed
    #eval_accumulation_steps=2,
    num_train_epochs=5,
    #max_steps=400,
    #warmup_steps=0,  # 1000,
    warmup_ratio=0.06,  #0.1, 0.06
    weight_decay=0.01,  #0.1,
    fp16=True,   # ! only makes sense at batch-size > 8. loads two copies of model weights, which creates overhead. https://huggingface.co/transformers/performance.html?#fp16
    fp16_full_eval=True,
    eval_strategy="epoch",
    seed=1,
    #load_best_model_at_end=True,
    #metric_for_best_model="accuracy",
    #eval_steps=50,  # evaluate after n steps if evaluation_strategy!='steps'. defaults to logging_steps
    save_strategy="no",  # options: "no"/"steps"/"epoch"
    #save_steps=100,  # Number of updates steps before two checkpoint saves.
    dataloader_num_workers = 12,
    #save_total_limit=1,  # If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir
    #logging_strategy="epoch",
    #report_to="all",  # "all"
    #run_name=run_name,
    #push_to_hub=True,  # does not seem to work if save_strategy="no"
    #hub_model_id=hub_model_id,
    #hub_token=config.HF_ACCESS_TOKEN,
    #hub_strategy="end",
    #hub_private_repo=True,
)

In [100]:
def compute_metrics_standard(eval_pred, label_text_alphabetical=list(model.config.id2label.values())):
    labels = eval_pred.label_ids
    pred_logits = eval_pred.predictions
    preds_max = np.argmax(pred_logits, axis=1)  # argmax on each row (axis=1) in the tensor

    # metrics
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds_max, average='micro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
    acc_balanced = balanced_accuracy_score(labels, preds_max)
    acc_not_balanced = accuracy_score(labels, preds_max)
    mcc = matthews_corrcoef(labels, preds_max)

    metrics = {'MCC': mcc,
            'f1_macro': f1_macro,
            'f1_micro': f1_micro,
            'accuracy_balanced': acc_balanced,
            'accuracy': acc_not_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'precision_micro': precision_micro,
            'recall_micro': recall_micro,
            #'label_gold_raw': labels,
            #'label_predicted_raw': preds_max
            }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} )  # print metrics but without label lists
    print("Detailed metrics: ", classification_report(
        labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical, sample_weight=None,
        digits=2, output_dict=True, zero_division='warn'),
    "\n")

    return metrics

In [101]:
trainer = Trainer(
    model=model,
    #model_init=model_init,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dstok['train'],
    eval_dataset=dstok['validation'],
    compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(model.config.id2label.values()))  #compute_metrics,
    #data_collator=data_collator,  # for weighted sampling per dataset; for dynamic padding probably not necessary because done by default  https://huggingface.co/course/chapter3/3?fw=pt
)

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [102]:
trainer.train()
#trainer.train(resume_from_checkpoint = 'training_base/checkpoint-157664')

Epoch,Training Loss,Validation Loss,Mcc,F1 Macro,F1 Micro,Accuracy Balanced,Accuracy,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,No log,0.565873,0.000000,0.422997,0.733092,0.500000,0.733092,0.366546,0.500000,0.733092,0.733092
2,No log,0.647523,0.000000,0.422997,0.733092,0.500000,0.733092,0.366546,0.500000,0.733092,0.733092
3,No log,0.894894,0.000000,0.422997,0.733092,0.500000,0.733092,0.366546,0.500000,0.733092,0.733092
4,No log,0.813769,0.000000,0.422997,0.733092,0.500000,0.733092,0.366546,0.500000,0.733092,0.733092
5,No log,0.872920,0.000000,0.422997,0.733092,0.500000,0.733092,0.366546,0.500000,0.733092,0.733092


C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Aggregate metrics:  {'MCC': 0.0, 'f1_macro': 0.4229967232648198, 'f1_micro': 0.73309241094476, 'accuracy_balanced': 0.5, 'accuracy': 0.73309241094476, 'precision_macro': 0.36654620547238, 'recall_macro': 0.5, 'precision_micro': 0.73309241094476, 'recall_micro': 0.73309241094476}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\1338913006.py:27: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mikeb\.conda\envs\san

Detailed metrics:  {'LABEL_0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 517.0}, 'LABEL_1': {'precision': 0.73309241094476, 'recall': 1.0, 'f1-score': 0.8459934465296396, 'support': 1420.0}, 'accuracy': 0.73309241094476, 'macro avg': {'precision': 0.36654620547238, 'recall': 0.5, 'f1-score': 0.4229967232648198, 'support': 1937.0}, 'weighted avg': {'precision': 0.5374244829848007, 'recall': 0.73309241094476, 'f1-score': 0.6201913753598803, 'support': 1937.0}} 



...

C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Aggregate metrics:  {'MCC': 0.0, 'f1_macro': 0.4229967232648198, 'f1_micro': 0.73309241094476, 'accuracy_balanced': 0.5, 'accuracy': 0.73309241094476, 'precision_macro': 0.36654620547238, 'recall_macro': 0.5, 'precision_micro': 0.73309241094476, 'recall_micro': 0.73309241094476}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\1338913006.py:27: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mikeb\.conda\envs\san

Detailed metrics:  {'LABEL_0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 517.0}, 'LABEL_1': {'precision': 0.73309241094476, 'recall': 1.0, 'f1-score': 0.8459934465296396, 'support': 1420.0}, 'accuracy': 0.73309241094476, 'macro avg': {'precision': 0.36654620547238, 'recall': 0.5, 'f1-score': 0.4229967232648198, 'support': 1937.0}, 'weighted avg': {'precision': 0.5374244829848007, 'recall': 0.73309241094476, 'f1-score': 0.6201913753598803, 'support': 1937.0}} 



TrainOutput(global_step=125, training_loss=0.5637968139648437, metrics={'train_runtime': 489.4979, 'train_samples_per_second': 0.511, 'train_steps_per_second': 0.255, 'total_flos': 44065962885000.0, 'train_loss': 0.5637968139648437, 'epoch': 5.0})

# Covid

In [43]:
df = pd.read_csv('covid_tweets_labeled.csv')
df = df[['text', 'non_comp']]
df['hypothesis'] = 'The author of this tweet does not believe COVID is dangerous.'
df.rename({'text':'premise', 'non_comp':'entailment'}, axis = 1, inplace = True)
df['entailment'].replace({0:1, 1:0}, inplace = True)

C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\3143349128.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['entailment'].replace({0:1, 1:0}, inplace = True)


### Few Shot

In [47]:
training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    lr_scheduler_type= "linear",
    group_by_length=False,
    learning_rate = 2e-5,
    #learning_rate = 9e-6,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps = 1, 
    num_train_epochs=5,
    warmup_ratio=0.06,  
    weight_decay=0.01, 
    fp16=True,   
    fp16_full_eval=True,
    eval_strategy="no",
    seed=1,
    save_strategy="no",
    dataloader_num_workers = 12,
)

tokenizer = AutoTokenizer.from_pretrained(modname)

In [48]:
%%time
# Define a function to initialize the modelin the trainer. This will make results reproducible
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modname, num_labels = 2, ignore_mismatched_sizes=True)

# Define the number of samples (shots) and random seeds to use
shots = [10, 25, 50, 100]
seeds = [1,2,3,4,5,6,7,8,9,10]

# Initialize lists to store results
mcc_list = []
acc_list = []
shots_list = []

# Iterate through different shot sizes
for shot in shots:
    # Iterate through different random seeds
    for seed in seeds:
        # Sample training data based on current shot size and seed
        train = df.sample(shot, random_state = seed)
        # Create validation set with remaining instances
        val = df[~df.index.isin(train.index)]
        
        # Create a DatasetDict with train and validation splits
        ds = DatasetDict({'train': Dataset.from_pandas(train, preserve_index=False), 'validation':Dataset.from_pandas(val, preserve_index=False)})
        # Tokenize the dataset
        dstok = ds.map(tokenize_function, batched = True)
        # Rename 'entailment' column to 'label'
        dstok = dstok.rename_columns({'entailment':'label'})
        # Define label mapping
        id2label = {0: "entailment", 1: "not_entailment"}
        
        # Initialize the Trainer
        trainer = Trainer(
            model_init = model_init,
            tokenizer=tokenizer,
            args=training_args,
            train_dataset=dstok['train'],
            eval_dataset=dstok['validation'],
            compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(model.config.id2label.values()))
        )
        
        # Train the model
        trainer.train()
        # Make predictions on validation set
        res = trainer.predict(dstok['validation'])
        preds = np.argmax(res.predictions, axis=-1)
        
        # Calculate Matthews Correlation Coefficient
        mcc_res = matthews_corrcoef(val['entailment'], preds)
        mcc_list.append(mcc_res)
        # Calculate Accuracy
        acc_res = accuracy_score(val['entailment'], preds)
        acc_list.append(acc_res)
        # Store the current shot size
        shots_list.append(shot)
    
    # Print progress
    print(shot)

Map: 100%|██████████| 1977/1977 [00:00<00:00, 5577.52 examples/s]


Step,Training Loss


Aggregate metrics:  {'MCC': 0.5158637973867064, 'f1_macro': 0.7491694092490311, 'f1_micro': 0.8234699038947901, 'accuracy_balanced': 0.727720393946673, 'accuracy': 0.8234699038947901, 'precision_macro': 0.7921515425585368, 'recall_macro': 0.727720393946673, 'precision_micro': 0.8234699038947901, 'recall_micro': 0.8234699038947901}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\1915898394.py:87: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.7419354838709677, 'recall': 0.5217391304347826, 'f1-score': 0.6126526082130965, 'support': 529.0}, 'not_entailment': {'precision': 0.8423676012461059, 'recall': 0.9337016574585635, 'f1-score': 0.8856862102849656, 'support': 1448.0}, 'accuracy': 0.8234699038947901, 'macro avg': {'precision': 0.7921515425585368, 'recall': 0.727720393946673, 'f1-score': 0.7491694092490311, 'support': 1977.0}, 'weighted avg': {'precision': 0.8154942628083476, 'recall': 0.8234699038947901, 'f1-score': 0.8126286607169237, 'support': 1977.0}} 



...

Map: 100%|██████████| 1887/1887 [00:00<00:00, 5010.54 examples/s]
C:\Users\mikeb\.conda\envs\sandbox\Lib\site-packages\accelerate\accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Aggregate metrics:  {'MCC': 0.5477404233811167, 'f1_macro': 0.7702354583378024, 'f1_micro': 0.830948595654478, 'accuracy_balanced': 0.754407364688551, 'accuracy': 0.830948595654478, 'precision_macro': 0.7948220187857111, 'recall_macro': 0.754407364688551, 'precision_micro': 0.830948595654478, 'recall_micro': 0.830948595654478}


C:\Users\mikeb\AppData\Local\Temp\ipykernel_12104\1915898394.py:87: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


Detailed metrics:  {'entailment': {'precision': 0.7310513447432763, 'recall': 0.5885826771653543, 'f1-score': 0.6521264994547438, 'support': 508.0}, 'not_entailment': {'precision': 0.8585926928281461, 'recall': 0.9202320522117476, 'f1-score': 0.888344417220861, 'support': 1379.0}, 'accuracy': 0.830948595654478, 'macro avg': {'precision': 0.7948220187857111, 'recall': 0.754407364688551, 'f1-score': 0.7702354583378024, 'support': 1887.0}, 'weighted avg': {'precision': 0.8242572371698981, 'recall': 0.830948595654478, 'f1-score': 0.8247521001963843, 'support': 1887.0}} 

100
CPU times: total: 17min 53s
Wall time: 3h 18min 51s


In [49]:
res_df = pd.DataFrame({'n':shots_list, 'mcc':mcc_list})
res_df['accuracy'] = acc_list
res_df.groupby('n').mean()

,mcc,accuracy
n,,
10,0.388921,0.775974
25,0.502602,0.813863
50,0.558912,0.834899
100,0.572250,0.836142
